# Basic ROS Nodes

You are already familiar with ROS nodes from the Turtlesim exercises. Nodes are simply modular programs that can act on data, and pass information between each other. See the diagram on the [Offical ROS2 Docs](https://docs.ros.org/en/jazzy/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Nodes/Understanding-ROS2-Nodes.html) for a visual animation of information being passed between nodes.

In the Turtlesim exercise, you worked with two nodes: the turtlesim itself, and the teleop keyboard controller. In that system, the Turtlesim keeps track of all the turtles, and moves them according to data it recieves from the keyboard controller.

![TurtleNode.png](images/TurtleNode.png)

As you can see, `teleop_turtle` (the keyboard node) sends data to `turtlesim` about the speed that it wants turtle1 to move at (`turtle1/cmd_vel`).

Then, `turtlesim` sends data back about the status of `rotate_absolute` actions. Try it out now! Launch the turtlesim and turtle_teleop_key nodes, and monitor info passed between them with `ros2 topic echo /turtle1/cmd_vel`.

In [ ]:
!ros2 run turtlesim turtlesim_node

Run this in a separate terminal, not in Jupyter! You will not be able to input keypresses.
```bash
ros2 run turtlesim turtle_teleop_key
```

In [ ]:
!ros2 topic echo /turtle1/cmd_vel
# You may also want to run this in a separate terminal, so you can see all your latest output

## 

## Types of Connections

As we saw in the node graph, nodes are connected together to show how they transfer information together. In this tutorial, we'll be covering 3 types of connections.

1. [Topics](https://docs.ros.org/en/jazzy/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Topics/Understanding-ROS2-Topics.html) (Publisher/Subscribers)
2. [Services](https://docs.ros.org/en/jazzy/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Services/Understanding-ROS2-Services.html) (Server/Client)
3. [Actions](https://docs.ros.org/en/jazzy/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Actions/Understanding-ROS2-Actions.html) (Action Server/Client)

Now that we know what nodes are and how they interact with each other, lets write our own!

## Writing A Node

To write ROS2 Nodes in python, we will be using the (rclpy)[https://github.com/ros2/rclpy] python library.

The first thing we need to do, as always, is enter our workspace and make a package.

In [ ]:
# Initialize your workspace variables
workspace = '/home/ubuntu/learn_ros_ws'
%cd $workspace
package_name = 'basic_nodes'

In [ ]:
%%bash --no-raise-error -s "$package_name"
cd src

# Create a package
ros2 pkg create --build-type ament_python $1 --node-name orphan_node

Great! Now we have a package for you to work in. Let's look at default the node it autofilled with, the `orphan_node`.

```python
def main():
    print('Hi from basic_nodes.')


if __name__ == '__main__':
    main()
```

It's just a main function with a print statement! Remember, each node is simply a program, and the ROS2 part is how it talks to other programs.

You can build and run this node with the following commands:

In [ ]:
%%bash --no-raise-error

colcon build --packages-select basic_nodes
source install/local_setup.bash
ros2 run basic_nodes orphan_node

Now that we have basic node setup, lets make it do something useful!

## Topics

### Publisher Node